In [73]:
#import modules
import pandas as pd
import csv
from pprint import pprint
import json
import requests
import time
from api_keys import key

In [74]:
#Open first year of IPOS file for 2019

path19 = "ipos-2019.csv"
path20 = "ipos-2020.csv"
path21 = "ipos-2021.csv"

file19 = pd.read_csv(path19)
file20 = pd.read_csv(path20)
file21 = pd.read_csv(path21)
file19_20 = file19.append(file20)
file19_20_21 = file19_20.append(file21)

file19_20_21

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,12/30/2019,MKD,Molecular Data Inc.,5.38,$0.17,-96.81%
1,12/19/2019,MNPR,"Monopar Therapeutics, Inc.",8.00,$2.75,-65.63%
2,12/19/2019,INDO,Indonesia Energy Corporation Limited,11.00,$3.87,-64.82%
3,12/13/2019,CIIC,CIIG Merger,10.00,$21.60,21.60%
4,12/13/2019,OCFT,OneConnect Financial Technology Co. Ltd.,10.00,$1.71,-82.90%
...,...,...,...,...,...,...
1030,1/6/2021,STPC,Star Peak Corp. II,10.0,9.87,9.87%
1031,1/6/2021,PPGH,Poema Global Holdings Corp.,10.0,9.91,-0.90%
1032,1/5/2021,BENE,Benessere Capital Acquisition Corp.,10.0,10.20,2.00%
1033,1/5/2021,CRU,Crucible Acquisition Corporation,10.0,9.73,-2.70%


In [84]:
file_clean = file19_20_21.loc[file19_20_21["IPO Price"] != "-" , : ]

file_clean["IPO Price"] = pd.to_numeric(file_clean["IPO Price"])

file_clean

#Removing ipo's launched at price higher than $8; as budget limitation wouldn't allow to buy anything more expensive

ipo_10 = file_clean.reset_index(drop=True)
tickers = ipo_10["Symbol"]

tickers_2 = []
for ticker in tickers:
    tickers_2.append(ticker)
sets = [tickers_2[x:x+5] for x in range(0,len(tickers_2),5)]

print(sets[1])

['SPT', 'HCCO', 'BILL', 'EH', 'LIVK']


C:\Users\gpsuser\AppData\Local\Temp/ipykernel_12452/1392103596.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_clean["IPO Price"] = pd.to_numeric(file_clean["IPO Price"])


In [85]:
#api call

url = "https://api.polygon.io/v3/reference/tickers/"
#pprint(requests.get(url + "YAYO" + "?apiKey=" + key).json())


list_date = []
city = []
state = []
employees = []
sic_description = []
start_price = ipo_10["IPO Price"]

for i in range(len(sets)):
    time.sleep(60)
    for x in range(len(sets[i])):
            response = requests.get(url + sets[i][x] + "?apiKey=" + key).json()
            try:
                list_date.append(response['results']['list_date'])
            except:
                list_date.append("")
            try:
                city.append(response['results']['address']['city'])
            except:
                city.append("")
            try:
                state.append(response['results']['address']['state'])
            except:
                state.append("")
            try:    
                employees.append(response['results']["total_employees"])
            except:
                employees.append("")
            try:
                sic_description.append(response['results']['sic_description'])
            except:
                sic_description.append("")    
                pass
    
  


In [89]:
full_df = pd.DataFrame({"Ticker" : tickers_2,
                       "Listed Data" : list_date,
                       "City" : city,
                       "State" : state,
                       "Number of employees" : employees,
                       "Industry" : sic_description,
                       "IPO Price" : start_price})

full_df

,Ticker,Listed Data,City,State,Number of employees,Industry,IPO Price
0,MKD,2019-12-30,,,189,,5.38
1,MNPR,2019-12-19,WILMETTE,IL,10,PHARMACEUTICAL PREPARATIONS,8.00
2,INDO,2019-12-19,,,62,,11.00
3,CIIC,,,,,,10.00
4,OCFT,2019-12-13,,,3597,,10.00
...,...,...,...,...,...,...,...
1741,STPC,,,,,,10.00
1742,PPGH,2021-02-26,GRAND CAYMAN,E9,4,BLANK CHECKS,10.00
1743,BENE,2021-01-28,MIAMI,FL,2,BLANK CHECKS,10.00
1744,CRU,2021-02-25,LOUISVILLE,CO,3,BLANK CHECKS,10.00


In [92]:
full_df["Industry"].value_counts()

BLANK CHECKS                                                   569
                                                               564
PHARMACEUTICAL PREPARATIONS                                    138
SERVICES-PREPACKAGED SOFTWARE                                   68
BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)                 66
                                                              ... 
INDUSTRIAL & COMMERCIAL FANS & BLOWERS & AIR PURIFING EQUIP      1
SERVICES-ADVERTISING AGENCIES                                    1
RETAIL-FURNITURE STORES                                          1
SERVICES-EDUCATIONAL SERVICES                                    1
GAS & OTHER SERVICES COMBINED                                    1
Name: Industry, Length: 123, dtype: int64

In [94]:
# Push ipos new data frame to a csv file

full_df.to_csv("full_ipo_data.csv",
              encoding="utf-8", index=False, header=True)

In [ ]:
#the code ends here